# smart preprocessor

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Definir caminho para nossa pasta principal
base_path = '../data/processed/noise_database'

In [3]:
# usei 'os.path.join' para fazer os caminhos das subpastas
gaussian_path = os.path.join(base_path, 'gaussian')
salt_pepper_path = os.path.join(base_path, 'salt_and_pepper')

In [4]:
os.makedirs(base_path, exist_ok=True) # garante que o código não de erro se for rodado novamente
os.makedirs(gaussian_path, exist_ok=True)
os.makedirs(salt_pepper_path, exist_ok=True)

In [5]:
print(f'Pastas criadas em: {os.path.abspath(base_path)}')

Pastas criadas em: c:\Users\ianfe\OneDrive\Área de Trabalho\Projeto Genese\data\processed\noise_database


RUIDO GAUSSIANO

In [6]:
image_path = '../data/raw/hand_xray.jpeg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

In [7]:
num_images_generated = 50 #Vamos criar 50 imagens "sujas" para salvar no "banco de dados" e alimentar a IA

In [11]:
#Loop da geração das imagens
for i in range(num_images_generated):
    #Definir padrão dos ruídos
    mean = 0
    std_dev = 70

    gaussian_noise = np.random.normal(mean, std_dev, image.shape)
    noisy_image = cv2.add(image.astype(np.float64), gaussian_noise).astype(np.uint8)

    #Nome da imagem
    filename = f'gaussian_{i:03d}.png'

    #juntando o caminho das pastas
    save_path = os.path.join(gaussian_path, filename)

    cv2.imwrite(save_path, noisy_image)

print(f"Concluido, foram adicionadas {num_images_generated} imagens na pasta '{gaussian_path}'")

Concluido, foram adicionadas 50 imagens na pasta '../data/processed/noise_database\gaussian'


In [12]:
noise_percentage = 0.05

In [13]:
#LOOP
for i in range(num_images_generated):
    noisy_image = image.copy()

    #sal e pimenta
    num_salt = int( (noisy_image.size * noise_percentage) / 2)
    num_pepper = int( (noisy_image.size * noise_percentage) / 2)

    #sortear coordenadas
    salt_rows = np.random.randint(0, image.shape[0] - 1, num_salt)
    salt_cols = np.random.randint(0, image.shape[1] - 1, num_salt)
    noisy_image[salt_rows, salt_cols] = 255 #muda todos os pixels sorteadas para branco (255)

    pepper_rows = np.random.randint(0, image.shape[0] - 1, num_pepper)
    pepper_cols = np.random.randint(0, image.shape[1] - 1, num_pepper)
    noisy_image[pepper_rows, pepper_cols] = 0

    filename = f"salt_pepper_{i:03d}.png"
    save_path = os.path.join(salt_pepper_path, filename)
    cv2.imwrite(save_path, noisy_image)


In [14]:
features = []
labels = []

In [16]:
for filename in os.listdir(gaussian_path):
    image_path = os.path.join(gaussian_path, filename)
    image_gaussian = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    hist = cv2.calcHist([image_gaussian], [0], None, [256], [0, 256])
    cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    features.append(hist.flatten())
    labels.append(0)

for filename in os.listdir(salt_pepper_path):
    image_path = os.path.join(salt_pepper_path, filename)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    features.append(hist.flatten())
    labels.append(1)

features = np.array(features)
labels = np.array(labels)

print(f"Formato da matriz de features: {features.shape}")
print(f"Formato do vetor labels: {labels.shape}")

Formato da matriz de features: (200, 256)
Formato do vetor labels: (200,)


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [18]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

model = SVC(kernel='linear')
model.fit(x_train_scaled, y_train)

predictions = model.predict(x_test_scaled)

accuracy = accuracy_score(y_test, predictions)

print(f"O robo acertou {accuracy * 100:.2f}%")

print("-" * 40)
for i in range(10):
    print(f"{predictions[i]} \t\t| {y_test[i]}")

O robo acertou 100.00%
----------------------------------------
1 		| 1
0 		| 0
0 		| 0
1 		| 1
0 		| 0
0 		| 0
1 		| 1
1 		| 1
1 		| 1
0 		| 0
